In [1]:
import pandas as pd
import json
import requests
import nba_api as nba

### Extracting data from stats.nba.com API

In [2]:
lst = [x+"-"+y for x,y in list(zip([str(i) for i in range(1984,2018)],[str(x)[1:] if len(str(x)) == 3 else str(x) for x in range(85,119)]))]
df_performance = pd.DataFrame()
session = requests.Session()
session.headers.update({'User-Agent': 'Custom user agent'})
for i in lst:
    url = "https://stats.nba.com/stats/leagueleaders?LeagueID=00&PerMode=Totals&Scope=S&Season=" + i + "&SeasonType=Regular+Season&StatCategory=PTS"
    response = session.get(url)
    results = response.json()
    df_year = pd.DataFrame(results["resultSet"]["rowSet"],columns=results["resultSet"]["headers"])
    df_year["YEAR"] = i
    df_performance = pd.concat([df_performance,df_year])


In [3]:
df_performance.reset_index(inplace=True)

In [4]:
df_performance.drop(['index'],axis=1,inplace=True)

In [5]:
df_performance["YEAR"].value_counts()

2017-18    540
2014-15    492
2016-17    486
2013-14    482
2011-12    478
2015-16    476
2012-13    468
2004-05    464
2006-07    458
2005-06    457
2010-11    452
2007-08    451
2008-09    445
2003-04    442
2000-01    441
2009-10    441
2001-02    440
1999-00    438
1998-99    437
1997-98    435
1996-97    433
1995-96    429
2002-03    428
1994-95    406
1993-94    402
1992-93    389
1990-91    386
1991-92    385
1989-90    381
1988-89    353
1986-87    335
1987-88    332
1985-86    314
1984-85    309
Name: YEAR, dtype: int64

In [6]:
# Seeing different values for teams
df_performance['TEAM'].unique()

array(['CHI', 'BOS', 'DEN', 'ATL', 'GOS', 'DAL', 'PHL', 'KCK', 'MIL',
       'SAN', 'NYK', 'HOU', 'LAC', 'UTH', 'SEA', 'LAL', 'DET', 'NJN',
       'POR', 'CLE', 'WAS', 'IND', 'PHX', 'SAC', 'CHH', 'MIA', 'MIN',
       'ORL', 'TOR', 'VAN', 'UTA', 'GSW', 'PHI', 'SAS', 'MEM', 'NOH',
       'CHA', 'NOK', 'OKC', 'BKN', 'NOP'], dtype=object)

### Getting salaries from CSV from Data.world (https://data.world/datadavis/nba-salaries)

In [7]:
# First data set for players' salaries
df_salaries = pd.read_csv('datadavis-nba-salaries_salaries_1985to2018.csv')
df_salaries.head()

,league,player_id,salary,season,season_end,season_start,team
0,NBA,abdelal01,395000,1990-91,1991,1990,Portland Trail Blazers
1,NBA,abdelal01,494000,1991-92,1992,1991,Portland Trail Blazers
2,NBA,abdelal01,500000,1992-93,1993,1992,Boston Celtics
3,NBA,abdelal01,805000,1993-94,1994,1993,Boston Celtics
4,NBA,abdelal01,650000,1994-95,1995,1994,Sacramento Kings


In [8]:
df_salaries.groupby(['season']).count()

,league,player_id,salary,season_end,season_start,team
season,,,,,,
1984-85,210,210,210,210,210,206
1985-86,296,296,296,296,296,296
1986-87,40,40,40,40,40,40
1987-88,303,303,303,303,303,303
1988-89,321,321,321,321,321,321
1989-90,64,64,64,64,64,64
1990-91,353,353,353,353,353,353
1991-92,387,387,387,387,387,387
1992-93,404,404,404,404,404,404


In [9]:
# Second dataset containing details for the players. These details will be necessary downstream as we will need to create an identifier with data that matches with the dataset from stats.nba.com
df_players = pd.read_csv('datadavis-nba-salaries_players.csv')
df_players.head(3)

,_id,birthDate,birthPlace,career_AST,career_FG%,career_FG3%,career_FT%,career_G,career_PER,career_PTS,...,draft_pick,draft_round,draft_team,draft_year,height,highSchool,name,position,shoots,weight
0,abdelal01,"June 24, 1968","Cairo, Egypt",0.3,50.2,0.0,70.1,256,13.0,5.7,...,25th overall,1st round,Portland Trail Blazers,1990,6-10,"Bloomfield in Bloomfield, New Jersey",Alaa Abdelnaby,Power Forward,Right,240lb
1,abdulza01,"April 7, 1946","Brooklyn, New York",1.2,42.8,NaN,72.8,505,15.1,9.0,...,5th overall,1st round,Cincinnati Royals,1968,6-9,"John Jay in Brooklyn, New York",Zaid Abdul-Aziz,Power Forward and Center,Right,235lb
2,abdulka01,"April 16, 1947","New York, New York",3.6,55.9,5.6,72.1,1560,24.6,24.6,...,1st overall,1st round,Milwaukee Bucks,1969,7-2,"Power Memorial in New York, New York",Kareem Abdul-Jabbar,Center,Right,225lb


In [10]:
df_players.columns

Index(['_id', 'birthDate', 'birthPlace', 'career_AST', 'career_FG%',
       'career_FG3%', 'career_FT%', 'career_G', 'career_PER', 'career_PTS',
       'career_TRB', 'career_WS', 'career_eFG%', 'college', 'draft_pick',
       'draft_round', 'draft_team', 'draft_year', 'height', 'highSchool',
       'name', 'position', 'shoots', 'weight'],
      dtype='object')

In [11]:
# Merging the two data sets (salaries and player details)
df_ps =pd.merge(df_players, df_salaries, left_on = '_id', right_on = 'player_id')

In [12]:
df_ps.head()

,_id,birthDate,birthPlace,career_AST,career_FG%,career_FG3%,career_FT%,career_G,career_PER,career_PTS,...,position,shoots,weight,league,player_id,salary,season,season_end,season_start,team
0,abdelal01,"June 24, 1968","Cairo, Egypt",0.3,50.2,0.0,70.1,256,13.0,5.7,...,Power Forward,Right,240lb,NBA,abdelal01,395000,1990-91,1991,1990,Portland Trail Blazers
1,abdelal01,"June 24, 1968","Cairo, Egypt",0.3,50.2,0.0,70.1,256,13.0,5.7,...,Power Forward,Right,240lb,NBA,abdelal01,494000,1991-92,1992,1991,Portland Trail Blazers
2,abdelal01,"June 24, 1968","Cairo, Egypt",0.3,50.2,0.0,70.1,256,13.0,5.7,...,Power Forward,Right,240lb,NBA,abdelal01,500000,1992-93,1993,1992,Boston Celtics
3,abdelal01,"June 24, 1968","Cairo, Egypt",0.3,50.2,0.0,70.1,256,13.0,5.7,...,Power Forward,Right,240lb,NBA,abdelal01,805000,1993-94,1994,1993,Boston Celtics
4,abdelal01,"June 24, 1968","Cairo, Egypt",0.3,50.2,0.0,70.1,256,13.0,5.7,...,Power Forward,Right,240lb,NBA,abdelal01,650000,1994-95,1995,1994,Sacramento Kings


In [13]:
df_ps.groupby(['season'])['_id'].count()

season
1984-85    210
1985-86    296
1986-87     40
1987-88    303
1988-89    321
1989-90     64
1990-91    353
1991-92    387
1992-93    404
1993-94    394
1994-95    418
1995-96    388
1996-97    413
1997-98    444
1998-99    432
1999-00    526
2000-01    464
2001-02    459
2002-03    459
2003-04    458
2004-05    478
2005-06    494
2006-07    511
2007-08    486
2008-09    471
2009-10    472
2010-11    467
2011-12    468
2012-13    496
2013-14    410
2014-15    543
2015-16    527
2016-17    556
2017-18    551
Name: _id, dtype: int64

In [14]:
#to check for null_values in both data sets
null_cols = df_ps.isnull().sum()
null_cols

_id                0
birthDate          0
birthPlace         0
career_AST         0
career_FG%         0
career_FG3%        0
career_FT%         0
career_G           0
career_PER         0
career_PTS         0
career_TRB         0
career_WS          0
career_eFG%        0
college         1636
draft_pick      1902
draft_round     1902
draft_team      1902
draft_year      1902
height             0
highSchool       989
name               0
position           0
shoots             0
weight             0
league             0
player_id          0
salary             0
season             0
season_end         0
season_start       0
team               4
dtype: int64

In [15]:
null_display = df_ps[(df_ps['team'].isnull()==True)]
null_display

,_id,birthDate,birthPlace,career_AST,career_FG%,career_FG3%,career_FT%,career_G,career_PER,career_PTS,...,position,shoots,weight,league,player_id,salary,season,season_end,season_start,team
1419,bosweto01,"October 2, 1953","Montgomery, Alabama",1.6,53.3,54.5,72.4,366,13.7,7.7,...,Power Forward,Right,220lb,NBA,bosweto01,65000,1984-85,1985,1984,NaN
2200,cartwbi01,"July 30, 1957","Lodi, California",1.4,52.5,0.0,77.1,963,15.2,13.2,...,Center,Right,245lb,NBA,cartwbi01,600000,1984-85,1985,1984,NaN
13260,webstma01,"April 13, 1952","Baltimore, Maryland",1.2,48.8,28.6,62.1,617,13.8,7.0,...,Center,Right,225lb,NBA,webstma01,450000,1984-85,1985,1984,NaN
13802,williro02,"May 5, 1961","Houston, Texas",5.4,43.9,27.4,78.9,153,12.2,8.6,...,Point Guard,Right,175lb,NBA,williro02,120000,1984-85,1985,1984,NaN


In [16]:
# We checked on Google: even if they had a salary, these players did not play in these seasons for various reasons: injuries, playing for the reserves teams, etc. We decided to drop them.

df_ps.dropna(subset=['team'],inplace=True)

In [17]:
# Creating an identifier that allows us to match this dataframe with df_performance
df_ps['player_season_id'] = df_ps['name'] + df_ps['team'] + df_ps['season']

In [18]:
df_ps.head()

,_id,birthDate,birthPlace,career_AST,career_FG%,career_FG3%,career_FT%,career_G,career_PER,career_PTS,...,shoots,weight,league,player_id,salary,season,season_end,season_start,team,player_season_id
0,abdelal01,"June 24, 1968","Cairo, Egypt",0.3,50.2,0.0,70.1,256,13.0,5.7,...,Right,240lb,NBA,abdelal01,395000,1990-91,1991,1990,Portland Trail Blazers,Alaa AbdelnabyPortland Trail Blazers1990-91
1,abdelal01,"June 24, 1968","Cairo, Egypt",0.3,50.2,0.0,70.1,256,13.0,5.7,...,Right,240lb,NBA,abdelal01,494000,1991-92,1992,1991,Portland Trail Blazers,Alaa AbdelnabyPortland Trail Blazers1991-92
2,abdelal01,"June 24, 1968","Cairo, Egypt",0.3,50.2,0.0,70.1,256,13.0,5.7,...,Right,240lb,NBA,abdelal01,500000,1992-93,1993,1992,Boston Celtics,Alaa AbdelnabyBoston Celtics1992-93
3,abdelal01,"June 24, 1968","Cairo, Egypt",0.3,50.2,0.0,70.1,256,13.0,5.7,...,Right,240lb,NBA,abdelal01,805000,1993-94,1994,1993,Boston Celtics,Alaa AbdelnabyBoston Celtics1993-94
4,abdelal01,"June 24, 1968","Cairo, Egypt",0.3,50.2,0.0,70.1,256,13.0,5.7,...,Right,240lb,NBA,abdelal01,650000,1994-95,1995,1994,Sacramento Kings,Alaa AbdelnabySacramento Kings1994-95


## Back to working with the stats.nba.com API: the df_performance dataset

In [19]:
# The names of the teams in df_performance is in an abbreviation format ('CHI','LAL','BOS'), therefore not in the same format as in df_ps. We need to find a way to match them.
# This package has a dictionary of teams and their details. Among them, team full name and their abbreviations.

from nba_api.stats.static import teams
results = teams.get_teams()
active_franchises = pd.DataFrame(results)
active_franchises

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [20]:
# Old franchises are not provided, unfortunately. We need to insert them manually after quickly doing a Google search. We could have scraped a website, but we decided for manual update given the time constraints and the low amount of franchises we had to insert.
old_teams_abb_names = dict(zip(['CHH', 'GOS', 'KCK', 'NJN', 'NOH', 'NOK', 'PHL', 'SAN', 'SEA','UTH', 'VAN'],["Charlotte Hornets","Golden State Warriors","Kansas City Kings","New Jersey Nets","New Orleans Hornets","New Orleans/Oklahoma City Hornets","Philadelphia 76ers","San Antonio Spurs","Seattle Supersonics","Utah Jazz","Vancouver Grizzlies"]))

old_franchises = pd.DataFrame.from_dict(old_teams_abb_names, orient='index', columns=['full_name'])
old_franchises = old_franchises.reset_index()
old_franchises.rename(columns={'index':'abbreviation'},inplace=True)
old_franchises['id'] = [str(i) for i in range(0,11)]
old_franchises

franchises = pd.concat([active_franchises,old_franchises],axis=0)
franchises


,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949.0
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946.0
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970.0
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002.0
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966.0
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980.0
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976.0
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946.0
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967.0
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970.0


In [21]:
df_performance = pd.merge(df_performance,franchises, left_on = 'TEAM', right_on = 'abbreviation')
df_performance.rename(columns={'full_name':'team_full_name'},inplace=True)
df_performance[['PLAYER','YEAR','team_full_name']]

,PLAYER,YEAR,team_full_name
0,Michael Jordan,1984-85,Chicago Bulls
1,Orlando Woolridge,1984-85,Chicago Bulls
2,Quintin Dailey,1984-85,Chicago Bulls
3,Steve Johnson,1984-85,Chicago Bulls
4,Dave Corzine,1984-85,Chicago Bulls
...,...,...,...
14500,Solomon Hill,2017-18,New Orleans Pelicans
14501,Larry Drew II,2017-18,New Orleans Pelicans
14502,Walt Lemon Jr.,2017-18,New Orleans Pelicans
14503,Charles Cooke,2017-18,New Orleans Pelicans


In [22]:
# Creating the identifier in df_performance for the merge with df_ps
df_performance['player_season_id'] = df_performance['PLAYER'] + df_performance['team_full_name'] + df_performance['YEAR']

In [23]:
df = pd.merge(df_performance,df_ps, on = 'player_season_id', how = 'left')

In [24]:
df

,PLAYER_ID,RANK,PLAYER,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,...,position,shoots,weight,league,player_id,salary,season,season_end,season_start,team
0,893,1,Michael Jordan,CHI,82,3144,837,1625,0.515,9,...,Small Forward and Shooting Guard,Right,195lb,NBA,jordami01,550000.0,1984-85,1985.0,1984.0,Chicago Bulls
1,78615,14,Orlando Woolridge,CHI,77,2816,679,1225,0.554,0,...,Small Forward,Right,215lb,NBA,woolror01,300000.0,1984-85,1985.0,1984.0,Chicago Bulls
2,76497,49,Quintin Dailey,CHI,79,2101,525,1111,0.473,7,...,Shooting Guard,Right,180lb,NBA,dailequ01,410000.0,1984-85,1985.0,1984.0,Chicago Bulls
3,77134,116,Steve Johnson,CHI,74,1659,281,516,0.545,0,...,Center,Right,235lb,NBA,johnsst03,369000.0,1984-85,1985.0,1984.0,Chicago Bulls
4,76453,126,Dave Corzine,CHI,82,2062,276,568,0.486,0,...,Center,Right,250lb,NBA,corzida01,660000.0,1984-85,1985.0,1984.0,Chicago Bulls
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14501,203524,453,Solomon Hill,NOP,12,187,11,41,0.268,4,...,Small Forward,Right,225lb,NBA,hillso01,12236535.0,2017-18,2018.0,2017.0,New Orleans Pelicans
14502,203580,479,Larry Drew II,NOP,10,70,7,24,0.292,3,...,Point Guard,Right,180lb,NBA,drewla02,148318.0,2017-18,2018.0,2017.0,New Orleans Pelicans
14503,1627215,479,Walt Lemon Jr.,NOP,5,35,7,16,0.438,1,...,Point Guard,Left,180lb,NBA,lemonwa01,92160.0,2017-18,2018.0,2017.0,New Orleans Pelicans
14504,1628429,504,Charles Cooke,NOP,13,38,2,14,0.143,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df.drop(['city','_id','player_season_id','year_founded','state','id','season','draft_team','team','college','birthPlace','TEAM','shoots','player_id','league','nickname','draft_pick','draft_round','highSchool'], axis = 1,inplace=True)

In [26]:
df.to_csv('nba_player_performance_per_season_with_salaries.csv')